In [0]:
import re
import gensim
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [0]:
!pip install -U -q PyDrive
import os  
os.chdir('/content/')  
!git clone https://github.com/ruelj2/Google_drive.git  

from Google_drive.handle import Google_drive  
Gd = Google_drive()  

    100% |████████████████████████████████| 993kB 19.5MB/s 
Cloning into 'Google_drive'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 106 (delta 32), reused 38 (delta 9), pack-reused 0
Receiving objects: 100% (106/106), 23.33 KiB | 4.67 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [0]:
!pip install -U -q PyDrive ## you will have install for every colab session
from pydrive.auth import GoogleAuth     
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
Gd = Google_drive()  

In [0]:
json_import1 = drive.CreateFile({'id':'1DXnuatRulmjPV0T3lQPu8oyDw_2f34pK'})
json_import1.GetContentFile('new_file.txt')

In [0]:
file_ID='1DXnuatRulmjPV0T3lQPu8oyDw_2f34pK'
Gd.load_file('/content',file_ID)

new_file.txt loaded


In [0]:
def read_file(file_name):
	data_list=[]
	with open(file_name,"r") as f:
		for line in f:
			line=line.strip()
			label=' '.join((line[line.find("[")+1:line.find("]")]).replace(',','').split())
			text=line[line.find("'")+1:line.find("')")].strip()
			data_list.append([label,text])
	return data_list

In [0]:
file_name="new_file.txt"
psychExp_txt=read_file(file_name)

In [0]:
##consider n-gram features
def ngram(token,n):
  output=[]
  for i in range(n-1,len(token)):
    ngram=' '.join(token[i-n+1:i+1])
    output.append(ngram)
  return output  

In [0]:
def create_feature(text,nrange=(1,1)):
  text_features=[]
  text=text.lower()
  text_alphanum=re.sub('[^a-z0-9#]',' ',text)
  for n in range(nrange[0], nrange[1]+1):
    text_features += ngram(text_alphanum.split(),n)
  return(text_features)  

In [0]:
def convert_label(item, name): 
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()

In [0]:
emotions = ["😂", "😱", "😠", "😢", "😒", "😳", "😣"]

X_all = []
Y_all = []
for label, text in psychExp_txt:
    Y_all.append(torch.LongTensor(list(map(float,label.split()))))
    X_all.append(text)

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_all,Y_all,test_size=0.2,random_state=123) ### random_state = 123?

In [0]:
label_train=[]
label_test=[]
for i in Y_train:
  label_train.append(torch.nonzero(i).squeeze())
for i in Y_test:
  label_test.append(torch.nonzero(i).squeeze())

In [0]:
# build vocabulary 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_train)
vocab=vectorizer.get_feature_names()

In [0]:
sentence_train=[]
for i in X_train:
  b=[]
  i=create_feature(i)
  for j in i:
    try:
      b.append(vocab.index(j))
    except ValueError:
      vocab.append(j)
      b.append(len(vocab)-1)
  sentence_train.append(torch.LongTensor(b))

In [0]:
sentence_test=[]
for i in X_test:
  b=[]
  i=create_feature(i)
  for j in i:
    try:
      b.append(vocab.index(j))
    except ValueError:
      b
  sentence_test.append(torch.LongTensor(b))

In [0]:
def accuracy(test_text,test_label,Model,batch_size=1):
  total=len(test_label)
  correct=0
  for sentence,labels in zip(test_text,test_label):
    output=torch.argmax(Model(sentence))
    correct += (output==labels).sum()
  return float(correct)/total  

In [0]:
def metrics(test_text,test_label,Model,batch_size=1):
  a=np.array(test_label)
  b=np.zeros(len(test_label))
  for i,sentence in enumerate(test_text):
    predict=torch.argmax(Model(sentence))
    b[i]=predict
  F1=f1_score(a,b,average=None)
  F1_total = f1_score(a,b,average='macro')
  rscore=recall_score(a,b,average=None)
  rscore_total=recall_score(a,b,average='macro')
  pscore=precision_score(a,b,average=None)
  pscore_total=precision_score(a,b,average='macro')
  print("F1:"+str(F1))
  print("recall_score:"+str(rscore))
  print("precision_score:"+str(pscore))
  print("F1_toal:"+str(F1_total))
  print("rscore_total:"+str(rscore_total))
  print("pscore_total:"+str(pscore_total))

In [0]:
class CNNClassifier(nn.Module):
  def __init__(self,num_embeddings, embedding_dim, hidden_dim,target_size,kernel_size):
      super(CNNClassifier, self).__init__()
      self.kernel_size=kernel_size
      self.embedding_word=nn.Embedding(num_embeddings,embedding_dim)
      self.conv1=nn.Conv1d(embedding_dim,hidden_dim,kernel_size)
      self.conv2=nn.Conv1d(embedding_dim,hidden_dim,1)
      
      self.output=nn.Linear(hidden_dim,target_size)
    

  def forward(self,sentence):
    x=self.embedding_word(sentence).view(1,len(sentence),-1).transpose(1,2)
    if(len(sentence)>1):
      c=self.conv1(x)
      pool1=nn.MaxPool1d(len(sentence)-self.kernel_size+1)
    else:
      c=self.conv2(x)
      pool1=nn.MaxPool1d(1)
    c=F.relu(c)  
    p=pool1(c).view(1,-1)
    out=self.output(p)
    return out
    
    
    
    
    

In [0]:
### Train the CNN model
model=CNNClassifier(len(vocab),50,30,7,2)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=0)
epoch_accuracies = []
best_accuracy=0
for epoch in range(1000):
  i=0
  for sentence,labels in zip(sentence_train,label_train):
    i=i+1
    optimizer.zero_grad()
    outputs=model(sentence)
    labels=labels.unsqueeze(dim=0)
    loss=loss_function(outputs,labels)
    loss.backward()
    optimizer.step()
    if (i+1)%1000==0:
              print ('Epoch: [%d/%d], Step: [%d/%d] , Loss: %.4f, Dev: %.4f' 
                     % (epoch+1, 1000, i+1, len(sentence_train)//32, 
                        loss.data, accuracy(sentence_train,label_train,model)))
  epoch_accuracies.append(accuracy(sentence_test,label_test,model))
  if epoch_accuracies[-1] > best_accuracy:
    best_accuracy = epoch_accuracies[-1]
    best_epoch = epoch
    
  if(epoch>=10 and epoch_accuracies[-1]<=epoch_accuracies[-10]):
    break                
  # TODO: implement early stopping here      
  print(best_accuracy, best_epoch)
          

Epoch: [1/1000], Step: [1000/187] , Loss: 2.0960, Dev: 0.2286
Epoch: [1/1000], Step: [2000/187] , Loss: 1.0804, Dev: 0.2938
Epoch: [1/1000], Step: [3000/187] , Loss: 1.9428, Dev: 0.3219
Epoch: [1/1000], Step: [4000/187] , Loss: 0.9861, Dev: 0.3725
Epoch: [1/1000], Step: [5000/187] , Loss: 1.8771, Dev: 0.3972
0.36564171122994654 0
Epoch: [2/1000], Step: [1000/187] , Loss: 2.9138, Dev: 0.4051
Epoch: [2/1000], Step: [2000/187] , Loss: 0.3437, Dev: 0.4402
Epoch: [2/1000], Step: [3000/187] , Loss: 1.1175, Dev: 0.4330
Epoch: [2/1000], Step: [4000/187] , Loss: 0.6763, Dev: 0.4525
Epoch: [2/1000], Step: [5000/187] , Loss: 1.8149, Dev: 0.4631
0.38636363636363635 1
Epoch: [3/1000], Step: [1000/187] , Loss: 1.8063, Dev: 0.4549
Epoch: [3/1000], Step: [2000/187] , Loss: 0.1651, Dev: 0.4703
Epoch: [3/1000], Step: [3000/187] , Loss: 1.4583, Dev: 0.4855
Epoch: [3/1000], Step: [4000/187] , Loss: 1.0886, Dev: 0.4908
Epoch: [3/1000], Step: [5000/187] , Loss: 0.9196, Dev: 0.5037
0.3997326203208556 2
Epoch

In [0]:
metrics(sentence_test,label_test,model)

F1:[0.48648649 0.50869565 0.31602709 0.492      0.37113402 0.28762542
 0.3847981 ]
recall_score:[0.55450237 0.52232143 0.30701754 0.55156951 0.37305699 0.2160804
 0.37155963]
precision_score:[0.43333333 0.49576271 0.3255814  0.44404332 0.36923077 0.43
 0.39901478]
F1_toal:0.4066809664482492
rscore_total:0.41372969695457257
pscore_total:0.41385232991458704
